<a href="https://colab.research.google.com/github/cobang0111/DRL_2023_Fall/blob/main/stable_baseline3_SAC_sgminicat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pybullet==3.1.6 imageio-ffmpeg stable-baselines3

import os
import time
import math

import numpy as np
import imageio_ffmpeg
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from base64 import b64encode
from IPython.display import HTML
import gym
from gym import spaces

import pybullet
import pybullet_data
#from stable_baselines3.sac.policies import MlpPolicy
#from stable_baselines3 import SAC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 70.2 MB/s eta 0:00:00
  Created wheel for pybullet: filename=pybullet-3.1.6-cp310-cp310-linux_x86_64.whl size=97100139 sha256=f80793b381cb33f874cd3139e2e689fef677c21b149e768f67fc7757990b5095
  Stored in directory: /root/.cache/pip/wheels/15/df/4c/e18952b046b7398206b82f090f99e3b2f9a48a361a065cc6cd
Successfully built pybullet


In [2]:
!pip install 'shimmy>=0.2.1'

In [ ]:
from stable_baselines3.sac.policies import MlpPolicy
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
log_dir = "/tmp/gym/"
os.makedirs(log_dir, exist_ok=True)

# Simulation & Environment
FLAG = pybullet.URDF_USE_SELF_COLLISION_EXCLUDE_PARENT | pybullet.URDF_USE_SELF_COLLISION


# camera parameters
cam_target_pos = [0, -0.2, 0.2]
cam_distance = 2.5
cam_yaw, cam_pitch, cam_roll = 60, 0, 0
cam_width, cam_height = 480, 360
cam_up, cam_up_axis_idx, cam_near_plane, cam_far_plane, cam_fov = [0, 0, 0], 2, 0.01, 100, 60


class minicatEnv(gym.Env):
    def __init__(self, initial_position):
        super(minicatEnv, self).__init__()
        self.action_space = spaces.Box(low=-10, high=10, shape=(2, ))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(3, 4))
        pybullet.connect(pybullet.DIRECT) #Open directly without GUI
        pybullet.resetSimulation()
        pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
        pybullet.setTimeStep(1/400)
        self.plane = pybullet.loadURDF("plane.urdf")
        self.robot = pybullet.loadURDF("/content/ver8/urdf/ver8.urdf",[0,0,0],useFixedBase=1, flags = FLAG)
        self.previous_position = initial_position

    def position_update(self, new_position):
      difference = new_position - self.previous_position
      self.previous_position = new_position
      return difference

    def step(self, action):
        torque2 = action[0]
        torque3 = action[1]
        pybullet.setJointMotorControl2(self.robot, 2, pybullet.POSITION_CONTROL, force = torque2)
        pybullet.setJointMotorControl2(self.robot, 3, pybullet.POSITION_CONTROL, force = torque3)

        # Step the simulation
        pybullet.stepSimulation()

        # Get the joint's angle and velocity
        joint_info_0 = pybullet.getJointState(self.robot, 0)
        joint_info_1 = pybullet.getJointState(self.robot, 1)
        joint_info_2 = pybullet.getJointState(self.robot, 2)
        joint_info_3 = pybullet.getJointState(self.robot, 3)

        joint_angle = [pybullet.getJointState(self.robot, i)[0] for i in range(4)]
        joint_velocity = [pybullet.getJointState(self.robot, i)[1] for i in range(4)]
        joint_reaction = [pybullet.getJointState(self.robot, i)[2] for i in range(4)]
        joint_torque = [pybullet.getJointState(self.robot, i)[3] for i in range(4)]

        pos_diff_0 = self.position_update(joint_angle[0])
        pos_diff_1 = self.position_update(joint_angle[1])

        # Calculate reward
        reward = 1 * abs(pos_diff_1)

        # The episode is done if the pole is more than some degrees from vertical
        done = joint_angle[1] > np.radians(90)

        arr = [joint_angle, joint_velocity, joint_torque]

        return np.array(arr), reward, done, {}


    def reset(self):
        pybullet.connect(pybullet.DIRECT) #Open directly without GUI
        pybullet.resetSimulation()
        pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
        pybullet.setTimeStep(1/400)
        self.plane = pybullet.loadURDF("plane.urdf")
        self.robot = pybullet.loadURDF("/content/ver8/urdf/ver8.urdf",[0,0,0],useFixedBase=1, flags = FLAG)

        # Initializing
        pybullet.setGravity(0, 0, -9.81)
        pybullet.setJointMotorControl2(self.robot, 0, pybullet.VELOCITY_CONTROL, force = 0)
        pybullet.setJointMotorControl2(self.robot, 1, pybullet.VELOCITY_CONTROL, force = 0)
        pybullet.setJointMotorControl2(self.robot, 2, pybullet.VELOCITY_CONTROL, force = 0)
        pybullet.setJointMotorControl2(self.robot, 3, pybullet.VELOCITY_CONTROL, force = 0)
        pybullet.changeVisualShape(self.robot, 0, rgbaColor=[1, 1, 1, 1])
        pybullet.changeVisualShape(self.robot, 1, rgbaColor=[0.85, 0.7, 1, 1])
        pybullet.changeVisualShape(self.robot, 2, rgbaColor=[0.65, 0.5, 0.8, 1])
        pybullet.changeVisualShape(self.robot, 3, rgbaColor=[0.45, 0.3, 0.6, 1])

        # friction
        pybullet.changeDynamics(self.plane, -1, lateralFriction=0.5)
        #for joint_index in range(pybullet.getNumJoints(robot)):
            #pybullet.setJointMotorControl2(self.robot, joint_index, pybullet.POSITION_CONTROL, force = 0)
            #pybullet.changeDynamics(self.robot, joint_index, lateralFriction=0.2376545)
        pybullet.changeDynamics(self.robot, -1, lateralFriction=0.5)
        pybullet.changeDynamics(self.robot, 3, lateralFriction=0.5)

        initial_angle_0 = 0
        initial_angle_1 = np.radians(30)
        initial_angle_2 = np.radians(30)
        initial_angle_3 = np.radians(-30)
        pybullet.resetJointState(self.robot, 0, initial_angle_0) # Set the initial angle of the joint
        pybullet.resetJointState(self.robot, 1, initial_angle_1)
        pybullet.resetJointState(self.robot, 2, initial_angle_2)
        pybullet.resetJointState(self.robot, 3, initial_angle_3)

        joint_info_0 = pybullet.getJointState(self.robot, 0)  # Get the initial joint state
        joint_info_1 = pybullet.getJointState(self.robot, 1)
        joint_info_2 = pybullet.getJointState(self.robot, 2)
        joint_info_3 = pybullet.getJointState(self.robot, 3)

        joint_angle = [pybullet.getJointState(self.robot, i)[0] for i in range(4)]
        joint_velocity = [pybullet.getJointState(self.robot, i)[1] for i in range(4)]
        joint_reaction = [pybullet.getJointState(self.robot, i)[2] for i in range(4)]
        joint_torque = [pybullet.getJointState(self.robot, i)[3] for i in range(4)]

        arr = [joint_angle, joint_velocity, joint_torque]

        return np.array(arr)

    def render(self):
        cam_view_matrix = pybullet.computeViewMatrixFromYawPitchRoll(cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx)
        cam_projection_matrix = pybullet.computeProjectionMatrixFOV(cam_fov, cam_width*1./cam_height, cam_near_plane, cam_far_plane)
        image = pybullet.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]
        image = np.ascontiguousarray(image)
        return image

    def close(self):
        pybullet.disconnect()

# Create the environment
env = minicatEnv(initial_position = 0)

model = SAC('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=1000)
#plot_results(log_dir+'monitor.csv')
model.save("sac_minicat_env")

del model # remove to demonstrate saving and loading

model = SAC.load("sac_minicat_env")

vid = imageio_ffmpeg.write_frames('vid.mp4', (cam_width, cam_height), fps=400)
vid.send(None) # seed the video writer with a blank frame


# Enjoy trained agent
for i in range(1):
    obs = env.reset()
    for j in range(10000):
        action, _states = model.predict(obs)
        #print(action)
        obs, rewards, done, info = env.step(action)
        image = env.render()
        vid.send(image)
        if done:
            break

vid.close()
env.close()


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
# Play recorded video

mp4 = open('vid.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)